# Single selection

In [52]:
import altair as alt
import pandas as pd
import numpy as np

np.random.seed(0)

In [54]:


n_objects = 20
n_times = 50

# Create one (x, y) pair of metadata per object
locations = pd.DataFrame({
    'id': range(n_objects),
    'x': np.random.randn(n_objects),
    'y': np.random.randn(n_objects)
})

# Create a 50-element time-series for each object
timeseries = pd.DataFrame(np.random.randn(n_times, n_objects).cumsum(0),
                          columns=locations['id'],
                          index=pd.RangeIndex(0, n_times, name='time'))

# Melt the wide-form timeseries into a long-form view
timeseries = timeseries.reset_index().melt('time')

# Merge the (x, y) metadata into the long-form view
timeseries['id'] = timeseries['id'].astype(int)  # make merge not complain
data = pd.merge(timeseries, locations, on='id')

data.head()

,time,id,value,x,y
0,0,0,-0.401298,1.669251,0.209849
1,1,0,-1.505914,1.669251,0.209849
2,2,0,-0.775563,1.669251,0.209849
3,3,0,-2.256809,1.669251,0.209849
4,4,0,-2.712634,1.669251,0.209849


In [55]:

# Data is prepared, now make a chart

selector = alt.selection_single(empty='all', fields=['id'])

base = alt.Chart(data).properties(
    width=250,
    height=250
).add_selection(selector)

points = base.mark_point(filled=True, size=200).encode(
    x='mean(x)',
    y='mean(y)',
    color=alt.condition(selector, 'id:O', alt.value('lightgray'), legend=None),
)

timeseries = base.mark_line().encode(
    x='time',
    y=alt.Y('value', scale=alt.Scale(domain=(-15, 15))),
    color=alt.Color('id:O', legend=None)
).transform_filter(
    selector
)

points | timeseries

alt.HConcatChart(...)

# Interval selection

In [38]:
import altair as alt
from vega_datasets import data

source = data.cars()
brush = alt.selection(type='interval')

alt.Chart(source).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
).add_selection(brush)



alt.Chart(...)

# Interval selection

In [48]:
import altair as alt
import pandas as pd
import numpy as np

x = np.random.normal(size=100)
y = np.random.normal(size=100)

m = np.random.normal(15, 1, size=100)

source = pd.DataFrame({"x": x, "y":y, "m":m})

# gapminder = pd.read_csv('data/gapminder.csv')
# interval selection in the scatter plot
pts = alt.selection(type="interval", encodings=["x"])

# left panel: scatter plot
points = alt.Chart().mark_point(filled=True, color="blue").encode(
    x='LifeExpectancy',
    y='GDP'
).transform_filter(
    pts
).properties(
    width=300,
    height=300
)

# right panel: histogram
mag = alt.Chart().mark_bar().encode(
    x='mbin:N',
    y="count()",
    color=alt.condition(pts, alt.value("blue"), alt.value("lightgray"))
).properties(
    width=300,
    height=300
).add_selection(pts)

# build the chart:
alt.hconcat(
    points,
    mag,
    data=gapminder
).transform_bin(
    "mbin",
    field="Population",
    bin=alt.Bin(step=5000000)
)



alt.HConcatChart(...)

# Interval selection

In [49]:
import altair as alt
import pandas as pd
import numpy as np

# gapminder = pd.read_csv('data/gapminder.csv')

# multiple selection in the scatter plot
pts = alt.selection(type="interval")

# left panel: scatter plot
points = alt.Chart(gapminder).mark_point(filled=True).encode(
    x='LifeExpectancy:Q',
    y='GDP:Q',
    color=alt.condition(pts, 'Continent:N', alt.value('lightgray')),
).add_selection(pts).properties(
    width=300,
    height=300
)

# right panel: histogram
mag = alt.Chart().mark_bar().encode(
    x='mbin:N',
    y="count()",
    color=alt.value('blue')
).transform_filter(
    pts
).properties(
    width=300,
    height=300
)

# build the chart:
alt.hconcat(
    points,
    mag,
    data=gapminder
).transform_bin(
    "mbin",
    field="Population",
    bin=alt.Bin(step=5000000)
)

alt.HConcatChart(...)

# Interval selection

In [51]:
import altair as alt
from vega_datasets import data

source = data.seattle_weather()

scale = alt.Scale(domain=['sun', 'fog', 'drizzle', 'rain', 'snow'],
                  range=['#e7ba52', '#a7a7a7', '#aec7e8', '#1f77b4', '#9467bd'])
color = alt.Color('weather:N', scale=scale)

# We create two selections:
# - a brush that is active on the top panel
# - a multi-click that is active on the bottom panel
brush = alt.selection_interval(encodings=['x'])
click = alt.selection_multi(encodings=['color'])

# Top panel is scatter plot of temperature vs time
points = alt.Chart().mark_point().encode(
    alt.X('monthdate(date):T', title='Date'),
    alt.Y('temp_max:Q',
        title='Maximum Daily Temperature (C)',
        scale=alt.Scale(domain=[-5, 40])
    ),
    color=alt.condition(brush, color, alt.value('lightgray')),
    size=alt.Size('precipitation:Q', scale=alt.Scale(range=[5, 200]))
).properties(
    width=550,
    height=300
).add_selection(
    brush
).transform_filter(
    click
)

# Bottom panel is a bar chart of weather type
bars = alt.Chart().mark_bar().encode(
    x='count()',
    y='weather:N',
    color=alt.condition(click, color, alt.value('lightgray')),
).transform_filter(
    brush
).properties(
    width=550,
).add_selection(
    click
)

alt.vconcat(
    points,
    bars,
    data=source,
    title="Seattle Weather: 2012-2015"
)

alt.VConcatChart(...)

In [57]:
import altair as alt
from vega_datasets import data

source = data.movies.url

pts = alt.selection(type="single", encodings=['x'])

rect = alt.Chart(data.movies.url).mark_rect().encode(
    alt.X('IMDB_Rating:Q', bin=True),
    alt.Y('Rotten_Tomatoes_Rating:Q', bin=True),
    alt.Color('count()',
        scale=alt.Scale(scheme='greenblue'),
        legend=alt.Legend(title='Total Records')
    )
)

circ = rect.mark_point().encode(
    alt.ColorValue('grey'),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection')
    )
).transform_filter(
    pts
)

bar = alt.Chart(source).mark_bar().encode(
    x='Major_Genre:N',
    y='count()',
    color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).properties(
    width=550,
    height=200
).add_selection(pts)

chart = alt.vconcat(
    rect + circ,
    bar
).resolve_legend(
    color="independent",
    size="independent"
)

display(chart)


alt.VConcatChart(...)

In [58]:
chart.save('altair.html')

In [60]:
print(chart.to_json())

{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.0.2.json",
  "config": {
    "view": {
      "continuousHeight": 300,
      "continuousWidth": 400
    }
  },
  "resolve": {
    "legend": {
      "color": "independent",
      "size": "independent"
    }
  },
  "vconcat": [
    {
      "data": {
        "url": "https://vega.github.io/vega-datasets/data/movies.json"
      },
      "layer": [
        {
          "encoding": {
            "color": {
              "aggregate": "count",
              "legend": {
                "title": "Total Records"
              },
              "scale": {
                "scheme": "greenblue"
              },
              "type": "quantitative"
            },
            "x": {
              "bin": true,
              "field": "IMDB_Rating",
              "type": "quantitative"
            },
            "y": {
              "bin": true,
              "field": "Rotten_Tomatoes_Rating",
              "type": "quantitative"
            }
      